In [18]:
#2023.2.13 I could work late playing python and still felt engross.
import tkinter as tk
from tkinter import simpledialog

def show_text_dialog(string):
    root = tk.Tk()
    
    root.withdraw()

    text = simpledialog.askstring("Input",string, parent=root)

    return text
    
    root.destroy()

In [19]:
import datetime

class Record:
    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.subjects = []
        self.emotions = []
        
    def get_start_time(self):
        self.start_time = datetime.datetime.now()
    
    def get_end_time(self):
        self.end_time = datetime.datetime.now()
    
    def save_to_datafile(self, filename):
        with open(filename, "a") as f: #用a模式打开文件不会清除txt原有内容
            f.write(f"Start time: {self.start_time}\n")
            for subject in self.subjects:
                f.write('Subject: ' + subject[0] + f" time:{subject[1]}\n")
            for emotion in self.emotions:
                f.write('Emotion: ' + emotion[0] + f" time:{emotion[1]}\n")
            f.write(f"End time: {self.end_time}\n\n")
    
    def from_datafile(self, filename, records):
        with open(filename, "r") as f:
            lines = f.readlines()
        for line in lines:
            if line[0:10] == 'Start time':
                record = Record()
                start_time_str = line.split(": ")[1].strip()
                start_time = datetime.datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S.%f")
                record.start_time = start_time
            if line[0:8] == 'End time':
                end_time_str = line.split(": ")[1].strip()
                end_time = datetime.datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S.%f")
                record.end_time = end_time
                records.append(record) #将读到的record放到records队尾
                
    def emotion_det(self):
        text = show_text_dialog("How do you feel?") 
        self.emotions.append([text,datetime.datetime.now()])
        
    def subject_det(self):
        text = show_text_dialog("What do you want to do?")  
        self.subjects.append([text,datetime.datetime.now()])
        
    def display(self): #In matpolit
        pass
    
    def subject_syn(self): #syncronize subjects
        pass
                


In [20]:
def time_sub(end_hour,end_min,end_sec,sta_hour,sta_min,sta_sec):
    h_sub = end_hour-sta_hour
    m_sub = end_min-sta_min
    s_sub = end_sec-sta_sec
    if h_sub<0:
        h_sub = 24+h_sub
    if m_sub<0:
        m_sub = 60+m_sub
        h_sub = h_sub-1
    if s_sub<0:
        s_sub = 60+s_sub
        m_sub = m_sub-1
    return h_sub,m_sub,s_sub

In [21]:


import cv2
import datetime
import matplotlib.pyplot as plt
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW) #0表示默认camera
cap.open(0)
cap.set(3,320)
cap.set(4,240)


record = Record()
record.get_start_time()
while cap.isOpened():
    flag, frame = cap.read()
    
    now = datetime.datetime.now()
    hour_sub,min_sub,sec_sub = time_sub(now.hour,now.minute,now.second,record.start_time.hour,record.start_time.minute,record.start_time.second)
    time_str = '{:.0f}:{:.0f}:{:.0f}'.format(hour_sub,min_sub,sec_sub)
    cv2.putText(frame,time_str,(10,40),cv2.FONT_HERSHEY_SIMPLEX,0.6, (255,0,0), 2)

    cv2.imshow('Window',frame)
    
    
    key_pressed = cv2.waitKey(60)
    if key_pressed == 32: #Spacebar
        record.emotion_det()
    
    if key_pressed == 9: #Tab
        record.subject_det()
    
    if key_pressed == 27: #ESC
        break
    
    
record.get_end_time()
cap.release()
cv2.destroyAllWindows()
record.save_to_datafile('DataFile.txt')
